In [5]:
import json
import random
import sys
from pathlib import Path
import torch
import matplotlib.pyplot as plt
import numpy as np

# Add project source to path
project_root = Path.cwd().parent
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
from normalizer import DataNormalizer
from utils import load_config

plt.style.use('science.mplstyle')

# %%
# 2. LOAD ARTIFACTS
# --- Paths ---
CONFIG_FILE = project_root / "inputs/model_input_params.jsonc"
DATA_ROOT = project_root / "data"
config = load_config(CONFIG_FILE)
model_folder = DATA_ROOT / config["output_paths_config"]["fixed_model_foldername"]
normalized_data_folder = DATA_ROOT / config["data_paths_config"]["normalized_profiles_foldername"]
raw_data_folder = DATA_ROOT / config["data_paths_config"]["raw_profiles_foldername"]

# --- Load Model, Metadata, and Test Set list ---
device = torch.device("cpu")
print(f"Forcing all operations to use device: {device}")

model = torch.jit.load(model_folder / "best_model_jit.pt", map_location=device)
model.eval()
with (normalized_data_folder / "normalization_metadata.json").open("r") as f:
    norm_metadata = json.load(f)
with (model_folder / "test_set_info.json").open("r") as f:
    test_filenames = json.load(f)["test_filenames"]

# --- Get variable lists from config ---
species_vars = sorted(config["species_variables"])
global_vars = sorted(config["global_variables"])
species_labels = [s.replace('_evolution', '') for s in species_vars]

print(f"Setup complete. Model loaded on {device}.")

# %%
# 3. PREDICT AND PLOT FOR A SINGLE TEST PROFILE

# --- Select a random test file and load both versions ---
test_filename = random.choice(test_filenames)
with (normalized_data_folder / test_filename).open("r") as f:
    norm_profile = json.load(f)
with (raw_data_folder / test_filename).open("r") as f:
    raw_profile = json.load(f)

# --- Prepare the constant part of the input vector ---
initial_species_norm = [norm_profile[key][0] for key in species_vars]
global_conds = [norm_profile[key] for key in global_vars]
base_input = torch.tensor(initial_species_norm + global_conds, dtype=torch.float32)

# --- Predict ONLY for subsequent time steps (t > 0) ---
predicted_subsequent_norm = []
# MODIFICATION: Iterate starting from the second time step
for norm_time_step in norm_profile["t_time"][1:]:
    time_tensor = torch.tensor([norm_time_step], dtype=torch.float32)
    input_vector = torch.cat([base_input, time_tensor]).unsqueeze(0)
    
    with torch.no_grad():
        norm_pred = model(input_vector).squeeze(0)
        predicted_subsequent_norm.append(norm_pred)

# --- Combine known initial state with predicted subsequent states ---
# MODIFICATION: Create the full normalized evolution trajectory
initial_species_tensor = torch.tensor(initial_species_norm, dtype=torch.float32).unsqueeze(1)
if predicted_subsequent_norm:
    predicted_subsequent_tensor = torch.stack(predicted_subsequent_norm).T
    # Combine the known t=0 data with the predicted t>0 data
    full_evolution_norm = torch.cat([initial_species_tensor, predicted_subsequent_tensor], dim=1)
else:
    # Handle profiles with only one time step
    full_evolution_norm = initial_species_tensor

# --- Denormalize the complete trajectory at once ---
predicted_evolutions_denorm = []
for i, key in enumerate(species_vars):
    # The full_evolution_norm tensor now has the correct initial value
    series_tensor = full_evolution_norm[i]
    denorm_series = DataNormalizer.denormalize(series_tensor, norm_metadata, key)
    predicted_evolutions_denorm.append(denorm_series.numpy())

raw_times = raw_profile['t_time']

# --- Plot the results ---
fig, ax = plt.subplots(figsize=(14, 8))
colors = plt.cm.viridis(np.linspace(0, 1, len(species_vars)))

for i, key in enumerate(species_vars):
    ax.plot(raw_times, raw_profile[key], color=colors[i], label=species_labels[i] + " (True)")
    # The first point of the scatter plot will now be perfect
    ax.scatter(raw_times, predicted_evolutions_denorm[i], color=colors[i], marker='x', s=30)

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel("Time (s)", fontsize=14)
ax.set_ylabel("Species Abundance", fontsize=14)
ax.set_title(f"Predicted vs. True Species Evolution for {test_filename}", fontsize=16)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
#ax.set_ylim(1e-20, 2)
plt.tight_layout()
plt.show()

Forcing all operations to use device: cpu
Setup complete. Model loaded on cpu.
